In [1]:
# import the necessary libraries
from pyforest import *
import numpy_financial as npf
from dateutil.relativedelta import relativedelta
from datetime import date
# import streamlit as st #for frontend

In [1]:
from sepaxml import SepaDD
from sepaxml import SepaTransfer
import datetime, uuid
import pandas as pd 
import sys 

def int_to_decimal_str(integer):
    """
    Helper to convert integers (representing cents) into decimal currency
    string. WARNING: DO NOT TRY TO DO THIS BY DIVISION, FLOATING POINT
    ERRORS ARE NO FUN IN FINANCIAL SYSTEMS.
    @param integer The amount in cents
    @return string The amount in currency with full stop decimal separator
    """
    int_string = str(integer)
    if len(int_string) <= 2:
        return "0." + int_string.zfill(2)
    else:
        return int_string[:-2] + "." + int_string[-2:]


def decimal_str_to_int(decimal_string):
    """
    Helper to decimal currency string into integers (cents).
    WARNING: DO NOT TRY TO DO THIS BY CONVERSION AND MULTIPLICATION,
    FLOATING POINT ERRORS ARE NO FUN IN FINANCIAL SYSTEMS.
    @param string The amount in currency with full stop decimal separator
    @return integer The amount in cents
    """
    int_string = decimal_string.replace('.', '')
    int_string = int_string.lstrip('0')
    return int(int_string)


excel_in = pd.read_excel('Sepa_V.xlsx')
name = excel_in['Nom Prenom'][0]
iban = excel_in['IBAN'][0]
bic = excel_in['BIC'][0]
montant = decimal_str_to_int(str(excel_in['Montant'][0]))
descrip = excel_in['Description'][0]
exec = int(excel_in['Execution'][0])
file_name = excel_in['Nom fichier'][0]

# Credit Transfer (virement bancaire)

config = {
    "name": name,
    "IBAN": iban,
    "BIC": bic,
    "batch": True,
    # For non-SEPA transfers, set "domestic" to True, necessary e.g. for CH/LI
    "currency": "EUR",  # ISO 4217
}
sepa = SepaTransfer(config, clean=True)

payment = {
    "name": name,
    "IBAN": iban,
    "BIC": bic,
    "amount": montant,  # in cents
    "execution_date": datetime.date.today() + datetime.timedelta(days=exec), # 2 jours apres
    "description": descrip,
    # "endtoend_id": str(uuid.uuid1())  # optional
}
sepa.add_payment(payment)

#print(sepa.export(validate=True))
original_stdout = sys.stdout # Save a reference to the original standard output

with open(f"{file_name}.xml", 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(sepa.export(validate=True))
    sys.stdout = original_stdout # Reset the standard output to its original value'''

In [4]:
from sepaxml import SepaDD
from sepaxml import SepaTransfer
import datetime, uuid
import pandas as pd 

def int_to_decimal_str(integer):
    """
    Helper to convert integers (representing cents) into decimal currency
    string. WARNING: DO NOT TRY TO DO THIS BY DIVISION, FLOATING POINT
    ERRORS ARE NO FUN IN FINANCIAL SYSTEMS.
    @param integer The amount in cents
    @return string The amount in currency with full stop decimal separator
    """
    int_string = str(integer)
    if len(int_string) <= 2:
        return "0." + int_string.zfill(2)
    else:
        return int_string[:-2] + "." + int_string[-2:]

def decimal_str_to_int(decimal_string):
    """
    Helper to decimal currency string into integers (cents).
    WARNING: DO NOT TRY TO DO THIS BY CONVERSION AND MULTIPLICATION,
    FLOATING POINT ERRORS ARE NO FUN IN FINANCIAL SYSTEMS.
    @param string The amount in currency with full stop decimal separator
    @return integer The amount in cents
    """
    int_string = decimal_string.replace('.', '')
    int_string = int_string.lstrip('0')
    return int(int_string)
    

excel_in = pd.read_excel('Sepa_P.xlsx')
name = excel_in['Nom Prenom'][0]
iban = excel_in['IBAN'][0]
bic = excel_in['BIC'][0]
creditor_id= excel_in['Creditor ID'][0]
montant = decimal_str_to_int(str(excel_in['Montant'][0]))
descrip = excel_in['Description'][0]
mandate = excel_in['Mandate ID'][0]
exec = int(excel_in['Execution'][0])
type_p = excel_in['Type'][0]
mandate_d = excel_in['Date Mandat']
file_name = excel_in['Nom fichier'][0]

# Direct Debit (prelevement automatique)

config = {
    "name": name,
    "IBAN": iban,
    "BIC": bic,
    "batch": True,
    "creditor_id": creditor_id,  # supplied by your bank or financial authority
    "currency": "EUR",  # ISO 4217
    # "instrument": "B2B"  # - default is CORE (B2C)
}
sepa = SepaDD(config, schema="pain.008.001.02", clean=True)

payment = {
    "name": name,
    "IBAN": iban,
    "BIC": bic,
    "amount": montant,  # in cents
    "type": type_p,  # FRST,RCUR,OOFF,FNAL
    "collection_date": datetime.date.today() + datetime.timedelta(days=exec),
    "mandate_id": mandate, # trouver le mandate ID
    "mandate_date": datetime.date.today() + datetime.timedelta(days=0),
    "description": descrip,
    # "endtoend_id": str(uuid.uuid1())  # autogenerated if obmitted
}
sepa.add_payment(payment)

#print(sepa.export(validate=True))
original_stdout = sys.stdout # Save a reference to the original standard output

with open(f"{file_name}.xml", 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(sepa.export(validate=True))
    sys.stdout = original_stdout # Reset the standard output to its original value
